In [10]:
pip install MetaTrader5

In [214]:
import pandas as pd
import MetaTrader5 as mt

import time
from datetime import datetime, time as dt_time, timedelta
from pytz import timezone

In [250]:
if not mt.initialize():
    print("Error: Failed to initialize MetaTrader.")

In [253]:
from config import login, password, server

if not mt.login(login, password, server):
    print("Error: Failed to login.")

Error: Failed to login.


In [254]:
account_info = mt.account_info()
print(account_info)

if not account_info:
    print("Error: Account not found.")
else:
    login_number = account_info.login
    balance = account_info.balance
    equity = account_info.equity

    print()
    print(f"Login Number: {login_number}")
    print(f"Balance: {balance}")
    print(f"Equity: {equity}")

AccountInfo(login=24917667, trade_mode=2, leverage=100, limit_orders=5000, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=True, balance=100.0, credit=0.0, profit=0.0, equity=100.0, margin=0.0, margin_free=100.0, margin_level=0.0, margin_so_call=120.0, margin_so_so=100.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Aidan Ouckama', server='Forex.com-Live 536', currency='USD', company='Gain Capital Group, LLC')

Login Number: 24917667
Balance: 100.0
Equity: 100.0


In [225]:
# order request examples
volume = 0.01

# BUY position
buy_take_profit = 1.11175
buy_stop_loss = 1.11150
buy_request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_BUY,
    "price": mt.symbol_info_tick(SYMBOL).ask,
    "sl": buy_stop_loss, # FLOAT
    "tp": buy_take_profit, # FLOAT
    "comment": "Buy Position Opened.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

# SELL position
sell_stop_loss = 1.11175
sell_take_profit = 1.11150
sell_request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume,
    "type": mt.ORDER_TYPE_SELL,
    "price": mt.symbol_info_tick(SYMBOL).bid,
    "sl": sell_stop_loss,
    "tp": sell_take_profit,
    "comment": "Sell Position Opened.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

In [182]:
result = mt.order_send(buy_request)
position_id = result._asdict()['order']

In [ ]:
# order closing examples

# BUY order closing
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_SELL,
    "position": position_id, # id of the position we want to close
    "comment": "Buy Position Closed.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

# SELL order closing
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": SYMBOL,
    "volume": volume, # FLOAT
    "type": mt.ORDER_TYPE_BUY,
    "position": position_id, # id of the position we want to close
    "comment": "Sell Position Closed.",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_FOK,
}

close = mt.order_send(request)
print(close)

In [268]:
# constants
SYMBOL = "USDCAD!"
QTY = 0.01
FOREVER = True
ITERATIONS = 10
SL_PCT = 0.05
TP_PCT = 0.1

BUY_TYPE = 0
SELL_TYPE = 1

def order_type_to_str(order_type):
    """Converts numeric order type to its string representation."""
    if order_type == BUY_ORDER_TYPE:
        return "BUY"
    elif order_type == SELL_ORDER_TYPE:
        return "SELL"
    else:
        return "UNKNOWN"
    
def close_type_to_str(order_type):
    """Converts numeric close type to its string representation."""
    if order_type == BUY_ORDER_TYPE:
        return "SELL"
    elif order_type == SELL_ORDER_TYPE:
        return "BUY"
    else:
        return "UNKNOWN"

BUY_ORDER_TYPE = mt.ORDER_TYPE_BUY
SELL_ORDER_TYPE = mt.ORDER_TYPE_SELL

# utility
def get_current_market_times():
    """Fixes timezone handling for market times, including DST adjustment."""
    UTC_TIMEZONE = timezone("Etc/UTC")
    NYC_TIMEZONE = timezone("America/New_York")
    
    now_date = datetime.now(UTC_TIMEZONE).replace(second=0, microsecond=0)
    
    nyc_time = datetime.now(NYC_TIMEZONE)
    
    # adjust for DST and GMT offset (+2 hours for this use case)
    current_gmt = now_date + nyc_time.dst() + timedelta(hours=2)
    
    today_start_utc = datetime.combine(now_date.date(), dt_time.min).replace(tzinfo=UTC_TIMEZONE)
    today_gmt_start = today_start_utc + nyc_time.dst() + timedelta(hours=2)
    
    return today_gmt_start, current_gmt
    
def generate_ohlc(symbol):
    """Generates OHLC data for the given symbol and timeframe."""
    start_time, now = get_current_market_times()
    
    ohlc = pd.DataFrame(mt.copy_rates_range(symbol, mt.TIMEFRAME_M1, start_time, now))
    ohlc['time'] = pd.to_datetime(ohlc['time'], unit='s')
    return ohlc


def create_order_request(symbol, qty, order_type, price, sl, tp):
    """Creates a request dictionary for sending an order."""
    return {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": qty,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "Position Opened.",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }


def create_close_request(symbol, qty, order_type, price):
    position = mt.positions_get()[0]._asdict()['ticket']
    
    """Creates a request dictionary for closing an order."""
    return {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": qty,
        "type": order_type,
        "price": price,
        "position": position,
        "comment": "Position Closed.",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }


def place_order(symbol, qty, order_type, price, sl, tp):
    """Places a market order."""
    request = create_order_request(symbol, qty, order_type, price, sl, tp)
    return mt.order_send(request)


def close_order(symbol, qty, close_type, price):
    """Closes an existing position."""
    request = create_close_request(symbol, qty, close_type, price)
    return mt.order_send(request)


def add_deal(deals, order_type, price, qty):
    """Adds a deal to the list of deals."""
    deals.append({
        "type": order_type,
        "profit": None,
        "price": price,
        "qty": qty,
    })
    return deals


def calculate_deal_profit(deals, close_price, qty):
    """Calculates and updates the profit for the last deal."""
    last_deal = deals[-1]
    price_diff = close_price - last_deal['price']
    if last_deal['type'] == SELL_TYPE:
        price_diff *= -1  # reverse for sell orders
    
    profit = qty * price_diff
    last_deal['profit'] = round(profit, 2)  # round profit to 2 decimal places
    return deals

In [ ]:
def initialize_conditions(ohlc):
    """Extracts the current, last close, high, and low prices for conditions."""
    current_close = ohlc['close'].iloc[-1]
    last_close = ohlc['close'].iloc[-2]
    last_high = ohlc['high'].iloc[-2]
    last_low = ohlc['low'].iloc[-2]
    
    return current_close, last_close, last_high, last_low


def get_price_info():
    """Gets the current buy and sell prices and calculates stop-loss and take-profit."""
    buy_price = mt.symbol_info_tick(SYMBOL).ask
    sell_price = mt.symbol_info_tick(SYMBOL).bid

    buy_sl = buy_price * (1 - SL_PCT)
    buy_tp = buy_price * (1 + TP_PCT)
    sell_sl = sell_price * (1 + SL_PCT)
    sell_tp = sell_price * (1 - TP_PCT)
    
    return buy_price, sell_price, buy_sl, buy_tp, sell_sl, sell_tp


def open_position(deals, order_type, price, sl, tp):
    """Handles placing orders, updating deal information."""
    order_type_str = order_type_to_str(order_type)
    order_total = round(QTY * price, 2)
    
    place_order(SYMBOL, QTY, order_type, price, sl, tp)
    print(f"{order_type_str} Order Placed - {QTY} @ {price}; Total: {order_total}.")
    deals = add_deal(deals, order_type, price, QTY)
    
    return deals


def close_position(deals, close_type, price):
    """Handles placing and closing orders, updating deal information."""
    close_type_str = close_type_to_str(close_type)
    close_total = round(QTY * price, 2)
    
    close_order(SYMBOL, QTY, close_type, price)
    print(f"{close_type_str} Position Closed - {QTY} @ {price}; Total: {close_total}.")
    deals = calculate_deal_profit(deals, price, QTY)
    print(f"Profit Made From {close_type_str} Position: {deals[-1]['profit']}")
    
    return deals


def execute_strategy(deals):
    """Executes the trading strategy for each iteration."""
    i = ITERATIONS
    while FOREVER or i > 0:
        print(f"Iteration: {ITERATIONS - i + 1}")
        
        ohlc = generate_ohlc(SYMBOL)
        current_close, last_close, last_high, last_low = initialize_conditions(ohlc)
        buy_price, sell_price, buy_sl, buy_tp, sell_sl, sell_tp = get_price_info()

        long_condition = current_close > last_high
        short_condition = current_close < last_low
        closelong_condition = current_close < last_close
        closeshort_condition = current_close > last_close

        positions = mt.positions_get()
        no_positions = len(positions) == 0
        already_buy = already_sell = False
        try:
            if positions:
                pos_type = positions[0]._asdict()['type']
                already_buy = pos_type == BUY_TYPE
                already_sell = pos_type == SELL_TYPE
        except Exception as e:
            print(f"Error checking positions: {e}")

        if long_condition:
            if no_positions:
                deals = open_position(deals, BUY_ORDER_TYPE, buy_price, buy_sl, buy_tp)
            if already_sell:
                deals = close_position(deals, BUY_ORDER_TYPE, buy_price)
                deals = open_position(deals, BUY_ORDER_TYPE, buy_price, buy_sl, buy_tp)

        if short_condition:
            if no_positions:
                deals = open_position(deals, SELL_ORDER_TYPE, sell_price, sell_sl, sell_tp)
            if already_buy:
                deals = close_position(deals, SELL_ORDER_TYPE, sell_price)
                deals = open_position(deals, SELL_ORDER_TYPE, sell_price, sell_sl, sell_tp)
                
        if closelong_condition and already_buy:
            deals = close_position(deals, SELL_ORDER_TYPE, buy_price)
        elif closeshort_condition and already_sell:
            deals = close_position(deals, BUY_ORDER_TYPE, sell_price)
        
        i -= 1
        time.sleep(60)
    
    return deals


def close_remaining_positions(deals):
    """Closes any remaining open positions after the iterations are complete."""
    if len(mt.positions_get()) != 0:
        already_buy = mt.positions_get()[0]._asdict()['type'] == BUY_TYPE
        already_sell = mt.positions_get()[0]._asdict()['type'] == SELL_TYPE
        if already_buy:
            deals = close_position(deals, SELL_ORDER_TYPE, sell_price)
        if already_sell:
            deals = close_position(deals, BUY_ORDER_TYPE, buy_price)
    return deals


def print_summary(deals):
    """Prints a summary of all closed positions and the total profit."""
    if not deals:
        print("\nNo closed positions to summarize.")
        return
    
    df = pd.DataFrame([{
        'Type': deal['type'],
        'Profit': deal['profit'],
    } for deal in deals])

    total_profit = df['Profit'].sum()
    
    print("\nSummary of Closed Positions:")
    print(df)
    print(f"\nTotal Profit: {total_profit}")

# main execution
deals = []
deals = execute_strategy(deals)
deals = close_remaining_positions(deals)
print(deals)
print_summary(deals)

Iteration: 1
Iteration: 2
Iteration: 3
